# AI Product Photo Detector - Training on GPU

**Runtime > Change runtime type > T4 GPU** before running.

Training time: ~2-3 min (vs 45 min on CPU)

In [ ]:
# 1. Check GPU
!nvidia-smi
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

In [ ]:
# 2. Clone repo & install
!git clone https://github.com/nolancacheux/AI-Product-Photo-Detector.git
%cd AI-Product-Photo-Detector
!pip install -e ".[dev,ui]" -q
!pip install datasets -q

In [ ]:
# 3. Download dataset
!python scripts/download_cifake.py --max-per-class 2500

In [ ]:
# 4. Train with GPU
# The training script auto-detects CUDA
!python -m src.training.train --config configs/train_config.yaml

In [ ]:
# 5. Verify model was saved
import os
candidates = ["models/best_model.pt", "models/checkpoints/best_model.pt"]
model_path = next((p for p in candidates if os.path.exists(p)), None)
if model_path:
    size_mb = os.path.getsize(model_path) / (1024 * 1024)
    print(f"Model saved: {model_path} ({size_mb:.1f} MB)")
else:
    print("Model not found! Checking...")
    !find models/ -type f -name '*.pt'

In [ ]:
# 6. Quick test - load model and predict
import torch
from src.training.model import create_model
from PIL import Image
from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = create_model("efficientnet_b0", pretrained=False)
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Test on real and AI images
for label, path in [("real", "data/processed/test/real/"), ("ai", "data/processed/test/ai_generated/")]:
    img_file = sorted(os.listdir(path))[0]
    test_img = Image.open(os.path.join(path, img_file)).convert("RGB")
    input_tensor = transform(test_img).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(input_tensor)
        prob = torch.sigmoid(logits).item()
    pred = 'ai_generated' if prob > 0.5 else 'real'
    print(f"[{label:>4}] {img_file} -> {pred} (prob: {prob:.4f})")

In [ ]:
# 7. Download trained model to your machine
from google.colab import files
files.download(model_path)

In [ ]:
# 8. (Optional) Download MLflow artifacts
!zip -r mlflow_artifacts.zip mlruns/
files.download("mlflow_artifacts.zip")